In [ ]:
import os
import cv2
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class_names = ['flag_images' , 'noflag_images']
class_names_label = {class_name : i for i , class_name in enumerate(class_names)}
nb_classes = len(class_names)
print(class_names_label)
IMAGE_SIZE = (150,150)

def load_data(test_size=0.2):
    DIRECTORY = r"C:\flag"
    CATEGORY = ["flag_images" , "noflag_images"]
    images = []
    labels = []

    for category in CATEGORY:
        path = os.path.join(DIRECTORY , category)
        print("Loading {}".format(category))

        for file in os.listdir(path):
            label = class_names_label[category]
            img_path = os.path.join(path,file)

            # open and resize the img
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE)

            # Append the image and its corresponding label to the output
            images.append(image)
            labels.append(label) 

    images = np.array(images, dtype = 'float32')
    labels = np.array(labels , dtype = 'int32')

    return train_test_split(images, labels, test_size=test_size, random_state=42)

train_images, test_images, train_labels, test_labels = load_data()


data_augmentation = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(data_augmentation.flow(train_images, train_labels), epochs=10, validation_data=(test_images, test_labels))

def predict_image(model, img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMAGE_SIZE)
    img = np.array(img, dtype='float32')
    img = img.reshape((1,150,150,3))
    prediction = model.predict(img)
    return class_names[int(round(prediction[0][0]))]


image_path = r'C:\1\usa.png' # replace this with your image path
print(predict_image(model, image_path))


{'flag_images': 0, 'noflag_images': 1}
Loading flag_images
Loading noflag_images
Epoch 1/10
12/76 [===>..........................] - ETA: 57s - loss: 514.7363 - accuracy: 0.6017